In [1]:
from pathlib import Path
from collections.abc import Mapping

import torch
import ignite
import ignite.distributed as idist
from ignite.contrib.engines import common
from ignite.engine import Engine, Events, create_supervised_evaluator
from ignite.handlers import DiskSaver
from ignite.metrics import ConfusionMatrix, IoU, mIoU
from ignite.utils import setup_logger

import segmentation_models_pytorch as smp


In [2]:
import sys

from models.unets import unet_resnet
from dataflow.dataloaders import get_train_val_loaders
from dataflow.transforms import get_train_augmentation, get_validation_augmentation

In [3]:
model, preprocessing_function = unet_resnet('efficientnet-b0')

In [4]:
lr = 1e-3
optimizer = torch.optim.Adam(
    [
        {"params": model.encoder.parameters(), "lr": 1e-6},
        {"params": model.decoder.parameters(), "lr": lr / 10},
        {"params": model.segmentation_head.parameters(), "lr": lr},
    ],
    lr=5e-4,
)

In [5]:
num_epochs = 10

criterion = smp.utils.losses.DiceLoss()


In [6]:
debug=True

num_classes = 1

batch_size = 8 * idist.get_world_size()  # total batch size
val_batch_size = batch_size * 2
num_workers = 12
val_interval = 3
accumulation_steps = 4

val_img_size = 513
train_img_size = 480
data_dir = Path('/media/wwymak/Storage/spacenet/AOI_3_Paris_Train')
image_dir = data_dir / 'RGB-PanSharpen'
mask_dir = data_dir / 'masks'
summary_data_filepath = data_dir / 'summaryData' / 'AOI_3_Paris_Train_Building_Solutions.csv'
train_loader, val_loader, train_eval_loader = get_train_val_loaders(
    image_dir=image_dir,
    mask_dir=mask_dir,
    summary_data_filepath=summary_data_filepath,
    train_transforms=get_train_augmentation(),
    val_transforms=get_validation_augmentation(),
    train_ratio=0.8,
    batch_size=batch_size,
    num_workers=num_workers,
    limit_train_num_samples=100 if debug else None,
    limit_val_num_samples=100 if debug else None,
)

In [7]:
 trainer = create_supervised_trainer(model, optimizer, criterion, device=device)
trainer.logger = setup_logger("trainer")

NameError: name 'create_supervised_trainer' is not defined